In [27]:
import cv2
import json
import numpy as np
import os
import matplotlib.pyplot as plt

In [35]:
images=[]
masks=[]

In [29]:
def get_images(path_img, path_json):
    for dirname, _, filenames in os.walk(path_img):
        for filename in filenames:
            image = cv2.imread(os.path.join(dirname, filename))
            if image is None:
                print("error: ", os.path.join(dirname, filename))
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # res = cv2.resize(image, (224,224), interpolation = cv2.INTER_AREA)
            images.append(image)
        
            for dirname1, _, filenames1 in os.walk(path_json):
        
                for filename1 in filenames1:
                    if(filename1==filename.replace('leftImg8bit.png','gtFine_polygons.json')):
                        f = open(os.path.join(dirname1, filename1), 'r')
                        polygons = json.load(f)
                        f.close()
                        masks.append(polygons)

In [30]:
# function not used in the trainer but usefull to visualize the dataset
def mask_maker(objects, image):
    for i in range(len(objects)):
        if objects[i]['label']=='person':
            cv2.fillPoly(image, np.array([objects[i]['polygon']]), (0,0,255))
        elif objects[i]['label']=='car':
            cv2.fillPoly(image, np.array([objects[i]['polygon']]), (0,255,0))
        else:
            cv2.fillPoly(image, np.array([objects[i]['polygon']]), (255,0,0))

In [31]:
# function not used in the trainer but usefull to visualize the dataset
def boxes_drawer(objects, image):

    for i in range(len(objects)):
        rect=cv2.boundingRect(np.array([objects[i]['polygon']]))
        if objects[i]['label']=='person':
            cv2.rectangle(image, (rect[0],rect[1]), (rect[0]+rect[2],rect[1]+rect[3]), (0,0,255), 2)
        elif objects[i]['label']=='car':
            cv2.rectangle(image, (rect[0],rect[1]), (rect[0]+rect[2],rect[1]+rect[3]), (0,255,0), 2)
            

In [32]:
def object_list_maker(objects, image):
    cars=[]
    pedestrians=[]
    others=[]
    others_labels=[]

    vehicules_labels=['car','motorbike','bus','truck','cargroup','motocycle']
    human_labels=['person','rider']
    others_to_avoid_labels=['ground','out of roi','road','sidewalk']

    for i in range(len(objects)):
        rect=cv2.boundingRect(np.array([objects[i]['polygon']]))
        if rect[2]>=96 and rect[3]>=96:
            if objects[i]['label'] in vehicules_labels:
                cars.append(image[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]])
            elif objects[i]['label'] in human_labels:
                pedestrians.append(image[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]])
            elif objects[i]['label'] not in others_to_avoid_labels:
                others.append(image[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]])
                others_labels.append(objects[i]['label'])
    
    return cars, pedestrians, others, others_labels

In [33]:
def dataset_maker():
    global images
    global masks
    cars=[]
    pedestrians=[]
    others=[]
    others_labels=[]
    for i in range(len(images)):
        temp_cars, temp_pedestrians, temp_others, temp_others_labels =object_list_maker(masks[i]['objects'],images[i])
        cars.append(temp_cars)
        pedestrians.append(temp_pedestrians)
        others.append(temp_others)
        others_labels.append(temp_others_labels)
    del images
    del masks
    cars = list2d_to_1d(cars)
    pedestrians = list2d_to_1d(pedestrians)
    others = list2d_to_1d(others)
    others_labels = list2d_to_1d(others_labels)
    # list to numpy array
    cars_array=np.array(cars, dtype=object)
    pedestrians_array=np.array(pedestrians, dtype=object)
    others_array=np.array(others, dtype=object)
    others_labels_array=np.array(others_labels, dtype=object)
    return cars_array, pedestrians_array, others_array, others_labels_array

In [34]:
#2d list to 1d list
def list2d_to_1d(list):
    new_list=[]
    for i in range(len(list)):
        for j in range(len(list[i])):
            new_list.append(list[i][j])
    return new_list

In [ ]:
def resize_images(array, new_dim):
    resized_array = []
    for i in range(array.shape[0]):
        try:
            img = cv2.resize(array[i], new_dim)
        except:
            print('error ', i)
        else:
            resized_array.append(img)
    return np.array(resized_array)